In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
import os

In [2]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


In [5]:
ensemble_df = pd.concat([
    pd.read_csv(f'data/ensemble/{file}',index_col='PassengerId') for file in os.listdir('data/ensemble/')
],axis=1)
ensemble_df.head(2)

,KNN,LR
PassengerId,,
0001_01,0.64,0.849722
0002_01,0.08,0.084764


In [6]:
df = pd.concat([ensemble_df.loc[df_train.index],df_train[['Transported']]],axis=1)
df.head()

,KNN,LR,Transported
PassengerId,,,
0001_01,0.64,0.849722,False
0002_01,0.08,0.084764,True
0003_01,0.16,0.285502,False
0003_02,0.36,0.567600,False
0004_01,0.08,0.206688,True


# Cross Validation

In [7]:
X, y = df.drop('Transported',axis=1), df['Transported']
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [15]:
pipeline = Pipeline([
    ('model', (GradientBoostingClassifier(n_estimators=10000, n_iter_no_change=20, tol=1e-4, validation_fraction=0.1)))
])

In [16]:
params = {
     'model__max_depth': [3, 5, 7, 9],
     'model__learning_rate': [0.1,0.01],
     'model__subsample': [0.4,0.7,0.9],
}

In [17]:
grid = GridSearchCV(
    pipeline,
    param_grid=params,
    scoring='accuracy',
    cv=KFold(5, shuffle=True, random_state=42),
    verbose=3,
    return_train_score=True
)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END model__learning_rate=0.1, model__max_depth=3, model__subsample=0.4;, score=(train=0.793, test=0.772) total time=   0.1s
[CV 2/5] END model__learning_rate=0.1, model__max_depth=3, model__subsample=0.4;, score=(train=0.791, test=0.793) total time=   0.2s
[CV 3/5] END model__learning_rate=0.1, model__max_depth=3, model__subsample=0.4;, score=(train=0.792, test=0.773) total time=   0.1s
[CV 4/5] END model__learning_rate=0.1, model__max_depth=3, model__subsample=0.4;, score=(train=0.789, test=0.789) total time=   0.1s
[CV 5/5] END model__learning_rate=0.1, model__max_depth=3, model__subsample=0.4;, score=(train=0.791, test=0.785) total time=   0.2s
[CV 1/5] END model__learning_rate=0.1, model__max_depth=3, model__subsample=0.7;, score=(train=0.795, test=0.769) total time=   0.2s
[CV 2/5] END model__learning_rate=0.1, model__max_depth=3, model__subsample=0.7;, score=(train=0.786, test=0.793) total time=   0.1s
[CV 3/5

[CV 3/5] END model__learning_rate=0.01, model__max_depth=3, model__subsample=0.4;, score=(train=0.790, test=0.776) total time=   1.1s
[CV 4/5] END model__learning_rate=0.01, model__max_depth=3, model__subsample=0.4;, score=(train=0.785, test=0.794) total time=   1.0s
[CV 5/5] END model__learning_rate=0.01, model__max_depth=3, model__subsample=0.4;, score=(train=0.788, test=0.781) total time=   1.1s
[CV 1/5] END model__learning_rate=0.01, model__max_depth=3, model__subsample=0.7;, score=(train=0.792, test=0.775) total time=   1.5s
[CV 2/5] END model__learning_rate=0.01, model__max_depth=3, model__subsample=0.7;, score=(train=0.784, test=0.791) total time=   1.4s
[CV 3/5] END model__learning_rate=0.01, model__max_depth=3, model__subsample=0.7;, score=(train=0.791, test=0.777) total time=   1.7s
[CV 4/5] END model__learning_rate=0.01, model__max_depth=3, model__subsample=0.7;, score=(train=0.786, test=0.791) total time=   1.5s
[CV 5/5] END model__learning_rate=0.01, model__max_depth=3, mo

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('model',
                                        GradientBoostingClassifier(n_estimators=10000,
                                                                   n_iter_no_change=20))]),
             param_grid={'model__learning_rate': [0.1, 0.01],
                         'model__max_depth': [3, 5, 7, 9],
                         'model__subsample': [0.4, 0.7, 0.9]},
             return_train_score=True, scoring='accuracy', verbose=3)

In [18]:
pd.DataFrame(grid.cv_results_)[['mean_train_score','std_train_score','mean_test_score','std_test_score']].loc[grid.best_index_]

mean_train_score    0.789510
std_train_score     0.003926
mean_test_score     0.784297
std_test_score      0.008797
Name: 2, dtype: float64

In [19]:
model = grid.best_estimator_
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

       False       0.79      0.74      0.76       861
        True       0.76      0.80      0.78       878

    accuracy                           0.77      1739
   macro avg       0.77      0.77      0.77      1739
weighted avg       0.77      0.77      0.77      1739



In [27]:
model.steps[0][1].feature_importances_

array([0.02193097, 0.97806903])